# 02. print 함수로 로그 출력하기

- 로그 : 프로그램의 동작 상태 또는 이력이 출력된 텍스트

### 요청에 걸린 시간 출력하기

In [1]:
import time
import requests

PAGE_URL_LIST = [
    'http://example.com/1.page',
    'http://example.com/2.page',
    'http://example.com/3.page',
]

for page_url in PAGE_URL_LIST:
    res = requests.get(page_url, timeout=30)
    print(
        f'페이지 URL : {page_url}, HTTP 상태 : {res.status_code}, 처리 시간(초) : {res.elapsed.total_seconds()}'
    )
    
    time.sleep(1)

페이지 URL : http://example.com/1.page, HTTP 상태 : 404, 처리 시간(초) : 0.289719
페이지 URL : http://example.com/2.page, HTTP 상태 : 404, 처리 시간(초) : 0.275996
페이지 URL : http://example.com/3.page, HTTP 상태 : 404, 처리 시간(초) : 0.311603


## 로그 출력과 관련된 다양한 개선이 필요한 이유
- 매번 로그를 출력하면 수많은 출력 속에서 오류 로그를 찾기 힘듬
- `python cralwer.py | tee -a crawler.log` 명령어를 통해 로그를 표준 출력과 파일에 동시에 출력할 수 있음
- 아래 코드는 `requests.get()` 부분에서 오류가 일어나면 프로그램이 종료되어 open으로 열었던 파일이 close 되지 않아 버그가 발생할 수 있다는 문제점이 있음

In [ ]:
import json
import time
import requests

PAGE_URL_LIST = [
    'http://example.com/1.page',
    'http://example.com/2.page',
    'http://example.com/3.page',
]

def fetch_pages():
    '''
    페이지의 내용을 추출
    '''
    # 처리 기록 전용 로그 파일을 append 모드로 열기
    f_info_log = open('crawler_info.log', 'a')
    
    # 오류 기록 전용 로그 파일을 append 모드로 열기
    f_error_log = open('crawler_error.log', 'a')
    
    # 추출 내용을 저장할 딕셔너리
    page_contents = {}
    
    # 터미널에 처리 시작을 출력하고 로그 파일에도 메시지를 출력
    msg = '크롤링을 시작합니다.\n'
    print(msg)
    f_info_log.write(msg)
    
    for page_url in PAGE_URL_LIST:
        r = requests.get(page_url, timeout=30)
        try:
            r.raise_for_status() # 응답에 문제가 생기면 예외를 발생
        except requests.exceptions.RequestException as e:
            # requests와 관련된 예외가 발생하면 터미널과 오류 로그에 오류를 출력
            msg = f"[ERROR] {e}\n"
            print(msg)
            f_error_log.write(msg)
            continue # 예외가 발생하면 반복을 중지하는 것이 아닌 건너뛰기
            
        # 정상적으로 내용을 추출했다면 딕셔너리에 내용 저장
        page_contents[page_url] = r.text
        time.sleep(1)
        
    f_info_log.close()
    f_error_log.close()
    
    return page_contents

if __name__ == '__main__':
    page_contents = fetch_pages()
    f_page_contents = open('page_contents.json', 'w')
    json.dump(page_contents, f_page_contents, ensure_ascii=False)
    f_page_contents.close()

- 아래 코드처럼 open, close 대신 with 문을 통해 close 메서드 누수를 막을 수 있음

In [ ]:
import json
import time
import requests

PAGE_URL_LIST = [
    'http://example.com/1.page',
    'http://example.com/2.page',
    'http://example.com/3.page',
]

def fetch_pages():
    '''
    페이지의 내용을 추출
    '''
    # 처리 기록 전용 로그 파일과 오류 기록 전용 로그 파일을 append 모드로 열기
    with open('crawler_info.log', 'a') as f_info_log, \
         open('crawler_error.log', 'a') as f_error_log:
    
        # 추출 내용을 저장할 딕셔너리
        page_contents = {}

        # 터미널에 처리 시작을 출력하고 로그 파일에도 메시지를 출력
        msg = '크롤링을 시작합니다.\n'
        print(msg)
        f_info_log.write(msg)

        for page_url in PAGE_URL_LIST:
            r = requests.get(page_url, timeout=30)
            try:
                r.raise_for_status() # 응답에 문제가 생기면 예외를 발생
            except requests.exceptions.RequestException as e:
                # requests와 관련된 예외가 발생하면 터미널과 오류 로그에 오류를 출력
                msg = f"[ERROR] {e}\n"
                print(msg)
                f_error_log.write(msg)
                continue # 예외가 발생하면 반복을 중지하는 것이 아닌 건너뛰기

            # 정상적으로 내용을 추출했다면 딕셔너리에 내용 저장
            page_contents[page_url] = r.text
            time.sleep(1)

        return page_contents

if __name__ == '__main__':
    page_contents = fetch_pages()
    f_page_contents = open('page_contents.json', 'w')
    json.dump(page_contents, f_page_contents, ensure_ascii=False)
    f_page_contents.close()